# chat UI in fastHTML

> chat UI implemented in fastHTML

In [ ]:
#| default_exp chat_ui

In [ ]:
#| hide
from nbdev.showdoc import *

## Chat App initialization

Start by creating the chat application with `FastHTML`.

In [ ]:
#| export
import uvicorn
import uuid
from fasthtml.common import *
from typing import Callable

from llmcam.fn_to_fc import capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file
from llmcam.fn_to_fc import tool_schema, complete, form_msg
from llmcam.store import add_api_tools, add_function_tools, remove_tools
from llmcam.store import execute_handler_core, handler_schema
from llmcam.yolo import detect_objects
from llmcam.dtcam import cap
from llmcam.file_manager import list_image_files, list_detection_files
from llmcam.plotting import plot_object 

In [ ]:
#| export
# Set up base youtube live tools and session tools as simplified database
default_tools = [tool_schema(fn) for fn in (
    capture_youtube_live_frame_and_save, 
    ask_gpt4v_about_image_file,
    detect_objects,
    cap,
    list_image_files,
    list_detection_files,
    plot_object)]
session_tools = {}

In [ ]:
#| export
# Utility functions to manage tools per session
def prepare_handler_schemas(
    session_id: str,  # Session ID to use
    fixup: Callable = None,  # Optional function to fix up the execution
):
    return [
        handler_schema(function, service_name="toolbox_manager", fixup=fixup, session_id=session_id) for \
        function in [add_api_tools, add_function_tools, remove_tools]
    ]

def execute_handler(
    function_name: str,  # Name of the function to execute
    session_id: str,  # Session ID to use
    **kwargs,  # Additional arguments to pass to the function
):
    tools = session_tools[session_id]
    return execute_handler_core(tools, function_name, **kwargs)

In [ ]:
#| export
# Set up the app, including daisyui and tailwind for the chat component
hdrs = (picolink, 
        Script(src="https://cdn.tailwindcss.com"),
        Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@4.11.1/dist/full.min.css"),
        Style("p {color: black;}"),
        Style("li {color: black;}"),
        MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']))
app = FastHTML(hdrs=hdrs)

## Chat components

Basic chat UI components can include Chat Message and a Chat Input. For a Chat Message, the important attributes are the actual message (str) and the role of the message owner (user - boolean value whether the owner is the user, not the AI assistant).

In [ ]:
#| export
# Chat message component (renders a chat bubble)
def ChatMessage(
        msg: str,  # Message to display
        user: bool  # Whether the message is from the user or assistant
    ):  # Returns a Div containing the chat bubble
    # Set class to change displayed style of bubble
    content_class = "chat-bubble chat-bubble-primary" if user else ""
    content_class += " marked py-2"
    user_style = "background-color: #fff0c7;"
    return  Div(cls=f"chat chat-end py-4" if user else "py-4")(
                Div('User' if user else 'Assistant', cls="chat-header"),
                Div(
                    msg,
                    cls=content_class,
                    style=user_style if user else ""),
                Hidden(msg, name="contents"),  # Hidden field for submitting past contents to form
                Hidden("user" if user else "assistant", name="roles")  # Hidden field for submitting corresponding owners
            )

In [ ]:
show_doc(ChatMessage)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L55){target="_blank" style="float:right; font-size:smaller"}

### ChatMessage

>      ChatMessage (msg:str, user:bool)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | str | Message to display |
| user | bool | Whether the message is from the user or assistant |

For the chat input, set the name for submitting a new message via form.

In [ ]:
#| export
# The input field for the user message. Also used to clear the
# input field after sending a message via an OOB swap
def ChatInput():  # Returns an input field for the user message
    return Input(name='msg', id='msg-input', placeholder="Type a message",
                 cls="input input-bordered w-full rounded-l-2xl", 
                 hx_swap_oob='true'  # Re-render the element to remove submitted message
                )

In [ ]:
show_doc(ChatInput)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L76){target="_blank" style="float:right; font-size:smaller"}

### ChatInput

>      ChatInput ()

### Action Buttons

Simple actions for creating a new message from the user side.

In [ ]:
#| export
def ActionButton(
        content: str,  # Text to display on the button
        message: str = None  # Message to send when the button is clicked
    ):  # Returns a button with the given content

    return Form(
        hx_post="/",
        hx_target="#chatlist",
        hx_swap="beforeend",  # Location: just before the end of element
    )(
        Hidden(content if message is None else message, name="msg"),
        Button(
            content, 
            cls="btn btn-secondary rounded-2 h-fit", 
            style="background-color: #ffe485; color: black; border-color: #ffe485;")
    )

def ActionPanel():  # Returns a panel of action buttons
    return Div(
        P("Quick actions", cls="text-lg text-white"),
        ActionButton("Introduce your model GPT-4o"),
        ActionButton(
            "Extract information from a YouTube Live", 
            "Capture and extract information from a YouTube Live. Use the default link."),
        cls="flex flex-col h-fit gap-4 py-4 px-4"
    )

In [ ]:
show_doc(ActionButton)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L83){target="_blank" style="float:right; font-size:smaller"}

### ActionButton

>      ActionButton (content:str, message:str=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| content | str |  | Text to display on the button |
| message | str | None | Message to send when the button is clicked |

In [ ]:
show_doc(ActionPanel)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L100){target="_blank" style="float:right; font-size:smaller"}

### ActionPanel

>      ActionPanel ()

## Router

### Home page
The home page should contain our message list and the Chat Input. The main page can be extracted by accessing the index (root) endpoint.

In [ ]:
#| export
scroll_script = Script("""
  // Function to scroll to the bottom of an element
  function scrollToBottom(element) {
    element.scrollTop = element.scrollHeight;
  }

  // Reference the expanding element
  const expandingElement = document.getElementById('chatlist');

  // Observe changes to the element's content and scroll down automatically
  const observer = new MutationObserver(() => {
    scrollToBottom(expandingElement);
  });

  // Start observing the expanding element for changes
  observer.observe(expandingElement, { childList: true, subtree: true });
""")

In [ ]:
#| export
title_script = Script("""
    // Function to set the title of the page
    document.title = "LLMCam Chatbot";
""")

In [ ]:
#| export
@app.get('/')
def index():
    sidebar = Div(
        ActionPanel(),
        P("Conversations", cls="text-lg text-white px-4"),
        cls="w-[30vw] flex flex-col p-0",
        style="background-color: #332b18;"
    )
    page =  Div(cls="w-full flex flex-col p-0")(  # Main page
        Form(
            hx_post="/",  # Operation: some POST endpoint with function `send` 
            hx_target="#chatlist",  # Target: element with ID 'chatlist'
            hx_swap="beforeend",  # Location: just before the end of element
            cls="w-full flex flex-col px-24 h-[100vh]"
        )(
            # The chat list
            Div(id="chatlist", cls="chat-box overflow-y-auto flex-1 w-full mt-10 p-4")(
                # One initial message from AI assistant
                ChatMessage("Hello! I'm a chatbot. How can I help you today?", False),
            ),
            # Input form
            Div(cls="h-fit mb-5 mt-5 flex space-x-2 mt-2 p-4")(
                Group(
                    ChatInput(), 
                    Button("Send", cls="btn btn-primary rounded-r-2xl", style="background-color: #ffe485;"))
            ),
            scroll_script,
            title_script
        )   
    )
    return Main(
        sidebar,
        page, 
        title="Chatbot",
        data_theme="lemonade", 
        cls="h-[100vh] w-full relative flex flex-row items-stretch overflow-hidden transition-colors z-0 p-0",)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
cap_2024.11.28_07:37:43_unclear.jpg
[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
cap_2024.11.28_07:38:39_unclear.jpg


### Form submission

At submission, this function should:

- Extract the new and all previous chat history  
- Prompt & get answers from ChatGPT from all these messages  
- Return a new ChatMessage

In [ ]:
#| export

# Handle the form submission
@app.post('/')
def send(session, msg: str, contents: list[str] = None, roles: list[str] = None):
    global session_tools
    global execute_handler
    
    if "tool_session_id" not in session or session["tool_session_id"] not in session_tools:
        # Initialize tools in session tools and create a session ID
        session_id = str(uuid.uuid4())
        session["tool_session_id"] = session_id

        session_tools[session_id] = []
        session_tools[session_id].extend(default_tools)
        session_tools[session_id].extend(prepare_handler_schemas(session_id, execute_handler))
    else:
        session_id = session["tool_session_id"]
        
    # If no contents or roles are provided, set them to empty lists
    if not contents: contents = []
    if not roles: roles = []

    # Create chat messages from the provided contents and roles
    messages = [ form_msg(role, content) for role, content in zip(roles, contents) ]
    nof_old_msgs = len(messages) # Number of old messages
    messages.append(form_msg("user", msg))
    
    # Add the user's message to the chat history
    complete(messages, session_tools[session_id])
    responses = messages[nof_old_msgs:]  # Get only the new messages
    
    # Create chat messages from the responses
    chat_messages = [
        ChatMessage(res['content'], res['role'] == 'user') for res in responses if 'content' in res \
            if res['role'] in ['user', 'assistant'] and res['content'] is not None
    ]
    
    return (*chat_messages,
            ChatInput()) # And clear the input field via an OOB swap

### Static files

In case the user needs to display images, serves files from directory `../data`.

In [ ]:
#| export
# Serve files from the 'data' directory
@app.get("/data/{file_name:path}")
def get_file(file_name: str):
    """Serve files dynamically from the 'data' directory."""
    data_path = save_dir = os.getenv("LLMCAM_DATA", "../data")
    file_path = Path(data_path) / file_name
    if file_path.exists():
        return FileResponse(file_path)
    return {"error": f"File '{file_name}' not found"}

## Runner

In addition to the main app, an utility function is implemented to run the app just by importing and executing this function to a Python file.

In [ ]:
#| export
def llmcam_chatbot(
        host="0.0.0.0",  # The host to listen on
        port=5001,  # The port to listen on
    ):
    # Import app from chat_ui base module
    from llmcam.chat_ui import app

    # Initialize session tools and execute handler
    session_tools = {}
    globals()["session_tools"] = session_tools

    def execute_handler(
        function_name: str,  # Name of the function to execute
        session_id: str,  # Session ID to use
        **kwargs,  # Additional arguments to pass to the function
    ):
        tools = session_tools[session_id]
        return execute_handler_core(tools, function_name, **kwargs)
    
    globals()["execute_handler"] = execute_handler
    
    # Run application with uvicorn
    uvicorn.run(app, host=host, port=port, log_level="info")

In [ ]:
show_doc(llmcam_chatbot)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L215){target="_blank" style="float:right; font-size:smaller"}

### llmcam_chatbot

>      llmcam_chatbot (host='0.0.0.0', port=5001)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| host | str | 0.0.0.0 | The host to listen on |
| port | int | 5001 | The port to listen on |

For running while testing with Jupyter notebook, use the `JupyUvi` in `fasthtml` to run in separate thread.

In [ ]:
#| eval: false
from fasthtml.jupyter import *

server = JupyUvi(app=app)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
cap_2024.11.28_07:32:54_unclear.jpg


In [ ]:
#| eval: false
server.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()